In [16]:
import sys
sys.path.insert(1, '../../Src')
from utils.preprocessing import *

import pandas as pd
import os
PATH_DATASET = '../../dataset/'
PATH_RESULTS = '../../results/Demanda/dataset/'

pd.set_option('display.max_rows', 50)

In [9]:
df_train_original = pd.read_csv(os.path.join(PATH_DATASET,'train/train_converted.csv'))
df_test_original = pd.read_csv(os.path.join(PATH_DATASET,'test/test_converted.csv'))

df_train = df_train_original[['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK','Z_WEEK_DATE','Demanda']].groupby(['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK','Z_WEEK_DATE']).sum().reset_index()
df_test = df_test_original[['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK','Z_WEEK_DATE','Demanda']].groupby(['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK','Z_WEEK_DATE']).sum().reset_index()
df_train = df_train.merge(df_train_original[['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK','Z_MARCA','Z_DEPARTAMENTO']],on=['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK'],how='left')
df_test  = df_test.merge(df_test_original[['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK','Z_MARCA','Z_DEPARTAMENTO']],on=['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK'],how='left')

In [10]:
df_train['Z_WEEK_DATE'] = pd.to_datetime(df_train['Z_WEEK_DATE'], errors='coerce')
df_test['Z_WEEK_DATE'] = pd.to_datetime(df_test['Z_WEEK_DATE'], errors='coerce')

In [11]:
dates = (set(df_train['Z_WEEK'].unique()) | set(df_test['Z_WEEK'].unique()))#df_auxiliar['Z_WEEK'].unique()
dates = sorted(dates)

dict_dates = {}
for idx,date in enumerate(dates):
    dict_dates[date] =idx
df_train['date_block_num'] = df_train['Z_WEEK'].map(dict_dates)
df_test['date_block_num'] = df_test['Z_WEEK'].map(dict_dates)

In [12]:
df_test['Demanda'] = 0

In [13]:
df_train["item_id"] = df_train["Z_MODELO"].astype(str) +"|"+ df_train["Z_PUNTO_VENTA"].astype(str) +"|"+ df_train["Z_GAMA"].astype(str) 
df_test["item_id"]  = df_test["Z_MODELO"].astype(str) +"|"+ df_test["Z_PUNTO_VENTA"].astype(str) +"|"+ df_test["Z_GAMA"].astype(str) 

In [14]:
df_train.fillna(-1, inplace=True)

In [17]:
%timeit df_train.to_pickle(os.path.join(PATH_RESULTS,'df_train.pkl'))

1.54 s ± 359 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit df_test.to_pickle(os.path.join(PATH_RESULTS,'df_test.pkl'))

245 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
